# Fine-Tuning Flow - Stage 3: Reducing Salt-and-Pepper Noise and Enhancing Sharpness

In [35]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [36]:
# Define the paths
folder_path = '../data/subset'

In [37]:
# Load image paths
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """

    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")

    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]

    return image_paths


In [38]:
# Load and preprocess images
def load_and_preprocess_images(image_paths, resize_dim=(256, 256)):
    images = []
    image_ids = []

    # Initialize tqdm progress bar
    for path in tqdm(image_paths, desc="Loading and preprocessing images", unit="image"):
        img = cv2.imread(path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        # img_resized = cv2.resize(img_gray, resize_dim)  # Resize for consistency
        images.append(img_gray)
        image_ids.append(f'Image_{len(images)}')  # Assign image ID as Image_1, Image_2, etc.

    return images, image_ids


In [39]:
# Load all image file paths from the specified folder
image_paths_all = load_images_from_folder(folder_path)

# Load and preprocess all images
total_images, total_image_ids = load_and_preprocess_images(image_paths_all)

# Randomly select 10 images for experimentation
# experiment_indices = random.sample(range(len(total_images)), 10)
# test_images = [total_images[i] for i in experiment_indices]
# test_image_ids = [total_image_ids[i] for i in experiment_indices]


Loading and preprocessing images: 100%|██████████| 12/12 [00:02<00:00,  5.24image/s]


## Applying the best Flow and techniques from previous steps

In [40]:
# Define the best techniques with their best parameters
best_techniques = {
    "Noise Reduction": lambda img: cv2.medianBlur(img, 3),
    "Histogram Equalization": lambda img: cv2.equalizeHist(img),
    "Binarization": lambda img: cv2.adaptiveThreshold(
        img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    ),
}


In [41]:
# Function to apply the best flow and save all images
def apply_best_flow_and_save(images, image_ids, output_folder):
    # Create output directory if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    processed_images = []  # List to store processed images
    processed_image_ids = []  # List to store image IDs

    for img, img_id in zip(images, image_ids):
        processed_image = img.copy()

        # Apply each technique in sequence as per the best flow
        for step_name, technique_func in best_techniques.items():
            processed_image = technique_func(processed_image)

        # Save the processed image to the output directory
        output_path = os.path.join(output_folder, f"Best_Flow_Image_{img_id}.jpg")
        cv2.imwrite(output_path, processed_image)

        # Store processed image and its ID
        processed_images.append(processed_image)
        processed_image_ids.append(img_id)

    print(f"All images have been processed and saved in '{output_folder}'.")

    # Return the processed images and their IDs for later processing
    return processed_images, processed_image_ids

In [42]:
# Assuming `total_images` and `total_image_ids` are defined and contain the list of images and their IDs
processed_images, processed_image_ids = apply_best_flow_and_save(total_images, total_image_ids, "./Data/It2/Best_Flow_Images")

All images have been processed and saved in './Data/It2/Best_Flow_Images'.


In [43]:
# Image Characteristics Calculation Functions - from data understanding it2
def calculate_brightness(image):
    return np.mean(image)

def calculate_sharpness(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_contrast(image):
    return image.std()

def calculate_noise(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    noise = cv2.absdiff(image, blurred)
    return np.var(noise)

In [44]:
# Load the CSV file with the image statistics
# images_stats_path = "../data-understanding/images_stats.csv"  
# images_stats_df = pd.read_csv(images_stats_path)

In [45]:
# images_stats_df.columns

In [46]:
# images_stats_df.drop(['Skew','Line Spacing', 'Tables Detected', 'Resolution', 'Detected Elements','Texture', 'Patterns'],axis=1, inplace=True)

In [47]:
def image_statistics_table(images, imagesId):
    stats_data = {'Image': [],
                  'Brightness': [],
                  'Sharpness': [],
                  'Contrast': [],
                  'Noise': [],}

    for i, img in enumerate(images):
        stats_data['Image'].append(imagesId[i])
        stats_data['Brightness'].append(calculate_brightness(img))
        stats_data['Sharpness'].append(calculate_sharpness(img))
        stats_data['Contrast'].append(calculate_contrast(img))
        stats_data['Noise'].append(calculate_noise(img))
    # Create a DataFrame to store per-image statistics
    df = pd.DataFrame(stats_data)
    return df

In [48]:
images_stats_df = image_statistics_table(processed_images, processed_image_ids)
print("Image Statistics Table:")
images_stats_df

Image Statistics Table:


,Image,Brightness,Sharpness,Contrast,Noise
0,Image_1,184.064784,101614.709924,114.265810,1911.472467
1,Image_2,186.841469,100512.937644,112.848748,1923.411572
2,Image_3,183.869726,103863.664641,114.362161,1912.976895
3,Image_4,183.213323,102336.881509,114.683371,1890.281215
4,Image_5,199.461862,84133.322482,105.250845,1860.932980
5,Image_6,194.386007,89845.786314,108.547281,1895.014561
6,Image_7,192.806443,88849.927661,109.504879,1889.037366
7,Image_8,168.084150,109124.750432,120.868427,1773.559901
8,Image_9,191.851313,85170.015914,110.068881,1798.468646
9,Image_10,193.391814,90281.637005,109.153648,1849.512335


In [49]:
# Create folder to save images for comparison
output_folder = "./Data/It2/Fine_Tuned_Flow"
os.makedirs(output_folder, exist_ok=True)

In [50]:
# Functions for each step in the image processing pipeline
def non_local_means_denoising(image, h=5, templateWindowSize=7, searchWindowSize=21):
    """Apply Non-Local Means Denoising."""
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)


def clahe_histogram_equalization(image, clipLimit=2.0, tileGridSize=(8, 8)):
    """Apply CLAHE Histogram Equalization."""
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    return clahe.apply(image)


def adaptive_threshold(image, adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, blockSize=11, C=2):
    """Apply Adaptive Thresholding."""
    return cv2.adaptiveThreshold(image, 255, adaptiveMethod, cv2.THRESH_BINARY, blockSize, C)


def morphological_opening(image, kernel_size=(3, 3)):
    """Apply Morphological Opening to reduce salt-and-pepper noise."""
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)


In [51]:
# Evaluation function to assess improvement in characteristics
def advanced_evaluation(image, techniques_dict, original_stats):
    evaluation_results = {}

    for technique_name, technique_func in techniques_dict.items():
        # Apply the technique
        processed_image = technique_func(image)

        # Calculate characteristics for the processed image
        stats = {
            "Brightness": calculate_brightness(processed_image),
            "Sharpness": calculate_sharpness(processed_image),
            "Contrast": calculate_contrast(processed_image),
            "Noise": calculate_noise(processed_image),
        }

        # Normalize metrics to comparable ranges (between 0 and 1, roughly)
        stats_normalized = {
            "Brightness": stats["Brightness"] / 255,
            "Sharpness": stats["Sharpness"] / 1000,
            "Contrast": stats["Contrast"] / 255,
            "Noise": stats["Noise"] / 255,
        }

        # Normalize the original stats for comparison
        original_stats_normalized = {
            "Brightness": original_stats["Brightness"] / 255,
            "Sharpness": original_stats["Sharpness"] / 1000,
            "Contrast": original_stats["Contrast"] / 255,
            "Noise": original_stats["Noise"] / 255,
        }

        # Weights for each characteristic (to determine their importance)
        weights = {
            "Brightness": 1.0,  # Higher is better (rewarded if improved)
            "Sharpness": 1.0,    # Higher is better (rewarded if improved) but images were generally sharp already 
            "Contrast": 2.0,     # Higher is better (rewarded if improved) the levels of contrast were lower and obstructed details
            "Noise": -1.5,       # Lower is better (penalized if increased)
        }

        # Calculate score using normalized metrics and weights
        score = 0
        for metric, value in stats_normalized.items():
            original_value = original_stats_normalized.get(metric, 0)
            score += weights[metric] * (value - original_value)

        evaluation_results[technique_name] = {"Score": score, "Stats": stats}

    # Determine the best technique based on the highest score
    best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
    return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


In [52]:
# Applying the improved flow to all images
for img, img_id in zip(total_images, total_image_ids):
    original_stats = images_stats_df[images_stats_df['Image'] == img_id].iloc[0].to_dict()

    # Step 1: Noise Reduction
    noise_reduced_image = non_local_means_denoising(img, h=5, templateWindowSize=7, searchWindowSize=21)

    # Step 2: Histogram Equalization
    histogram_equalized_image = clahe_histogram_equalization(noise_reduced_image)

    # Step 3: Binarization
    binarized_image = adaptive_threshold(histogram_equalized_image, adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, blockSize=11, C=2)

    # Step 4: Morphological Opening to reduce salt-and-pepper noise
    final_image = morphological_opening(binarized_image, kernel_size=(3, 3))

    # Save the final processed image for comparison
    output_path = os.path.join(output_folder, f"Fine_Tuned_Flow_Image_{img_id}.jpg")
    cv2.imwrite(output_path, final_image)

    print(f"Saved fine-tuned processed image for {img_id} at {output_path}")

print("All images processed and saved in the Fine-Tuned Flow stage.")


Saved fine-tuned processed image for Image_1 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_1.jpg
Saved fine-tuned processed image for Image_2 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_2.jpg
Saved fine-tuned processed image for Image_3 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_3.jpg
Saved fine-tuned processed image for Image_4 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_4.jpg
Saved fine-tuned processed image for Image_5 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_5.jpg
Saved fine-tuned processed image for Image_6 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_6.jpg
Saved fine-tuned processed image for Image_7 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_7.jpg
Saved fine-tuned processed image for Image_8 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_8.jpg
Saved fine-tuned processed image for Image_9 at ./Data/It2/Fine_Tuned_Flow\Fine_Tuned_Flow_Image_Image_9.jpg
Saved fine-tuned pr

### Improved Flow Explanation

The improved flow incorporates advanced techniques and specific parameters to enhance the final image quality. Below, I explain each step in detail, the reasoning behind the technique choices, and how each step contributes to the improved output.

#### Final Steps and Techniques

##### 1. Noise Reduction (Non-Local Means Denoising)
- **Technique**: Non-Local Means Denoising (`h=5`, `templateWindowSize=7`, `searchWindowSize=21`)
- **Reasoning**:
  - **Improvement Needed**: Median Blur was previously used but had some limitations in preserving details. Non-Local Means proved better in reducing noise while maintaining important features.
  - **Why This Step**: Non-Local Means is effective for reducing noise without excessive blurring, which is vital for maintaining legibility in handwritten notes. The chosen parameters (`h=5`) provided a moderate filtering strength to reduce noise without losing essential information.
  - **Effect**: This step significantly reduced noise while retaining the original details, which made the subsequent steps more effective.

##### 2. Histogram Equalization (CLAHE)
- **Technique**: CLAHE (Contrast Limited Adaptive Histogram Equalization)
- **Reasoning**:
  - **Improvement Needed**: Standard Histogram Equalization often led to over-enhancement, amplifying noise in regions with fewer features.
  - **Why This Step**: CLAHE limits contrast enhancement to prevent over-amplification and enhances local regions for more balanced results. It is well-suited for non-uniform lighting conditions, typical in handwritten documents.
  - **Effect**: Applying CLAHE improved the contrast and made the handwriting clearer while avoiding artifacts or excessive noise. The text and table lines became more pronounced without compromising on clarity.

##### 3. Binarization (Adaptive Threshold)
- **Technique**: Adaptive Threshold (`adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C`, `blockSize=11`, `C=2`)
- **Reasoning**:
  - **Improvement Needed**: Binarization is crucial for separating text from the background, but uniform binarization sometimes created artifacts.
  - **Why This Step**: The Adaptive Gaussian Threshold method takes local illumination differences into account, ensuring that text is distinct across varying lighting conditions. The chosen parameters (`blockSize=11`, `C=2`) helped capture more local details and improve text segmentation.
  - **Effect**: This step effectively made the text and table lines stand out against the background, resulting in better segmentation of the document content.

##### 4. Morphological Operations (Morphological Opening)
- **Technique**: Morphological Opening (`kernel_size=(3, 3)`)
- **Reasoning**:
  - **Improvement Needed**: After the binarization step, salt-and-pepper noise became more noticeable, necessitating further cleanup.
  - **Why This Step**: Morphological Opening (erosion followed by dilation) is ideal for removing scattered noise without affecting text shapes. The small kernel size (`3x3`) was chosen to remove noise conservatively, ensuring minimal impact on handwriting and table lines.
  - **Effect**: This step significantly reduced noise, providing a cleaner image suitable for downstream tasks like OCR, while preserving thin lines and small handwriting details.

### Summary of Improvements in the Flow
The improved flow made use of advanced techniques that were fine-tuned through experimentation:

1. **Enhanced Noise Reduction**: Switching from Median Blur to Non-Local Means resulted in better preservation of details, while effectively reducing noise.
2. **Controlled Contrast Enhancement**: CLAHE offered a more refined way to enhance contrast without over-amplifying regions, avoiding excessive noise.
3. **Better Text Segmentation**: Adaptive Threshold with tuned parameters provided better separation of text from the background, which was crucial for handling variations in lighting.
4. **Effective Noise Reduction Post-Binarization**: Morphological Opening helped in reducing salt-and-pepper noise while retaining important features.
